# State-dependent modulation of spiny projection neurons controlslevodopa-induced dyskinesia in a mouse model of Parkinson’s disease

**This Dataset (DANDI:001538):**
This dataset contains neurophysiology data from the Surmeier lab studying levodopa-induced dyskinesia (LID) in Parkinson's disease models. The data spans multiple experimental approaches:

- **Electrophysiology**: Patch-clamp recordings from striatal neurons (dSPNs and iSPNs)
- **Two-photon imaging**: Dendritic excitability and spine density measurements  
- **Behavioral analysis**: Abnormal involuntary movements (AIMs) and rotational behavior
- **Pharmacology**: Effects of various receptor agonists and antagonists
- **Optogenetics**: Selective stimulation of specific neuron populations

One of the nice features of dandi is that the data can be streamed directly anywhere from the world without download (although data can be loaded as well). To enable this streaming workflow we need to instantiate a dandi client.


In [ ]:
import os

from dandi.dandiapi import DandiAPIClient
from dotenv import load_dotenv

# Load environment variables from .env file
# This is required at the moment because the data is on draft status
load_dotenv()
# Load token from environment variable
token = os.getenv("DANDI_API_TOKEN")
if not token:
    raise ValueError("DANDI_API_TOKEN environment variable not set. Please set it with your DANDI API token.")

dandiset_id = "001538"
client = DandiAPIClient(token=token)
client.authenticate(token=token)

dandiset = client.get_dandiset(dandiset_id, "draft")
assets = dandiset.get_assets()
assets_list = list(assets)

## Fetching data

Dandisets contain **Assets** which are individual data files within the dataset. For this project, each asset represents one NWB file containing:
- **One experimental session** from **one subject** (animal)
- All data streams recorded during that session (voltage, imaging, behavior)
- Complete experimental metadata and protocols
- Standardized data organization following NWB format

In general, assets can be fetched by an id or by the path. The path of an NWB file (asset) on DANDI is of the form:

`sub-<subject_id>/sub-<subject_id>_ses-<session_id>_[desc-<description>]_<modalities>.nwb`

Which includes the session_id. For this dataset, the session_id was encoded with the following pattern:

`<figure>++<measurement>++<cell_type>++<state>++<pharmacology>++<genotype>++<timestamp>`

For example, `"F3++SomExc++iSPN++OffState++none++WT++20160523154318"` indicates:
- Figure 3 data
- Somatic excitability measurement  
- Indirect pathway SPNs (iSPN)
- Off-state (no levodopa)
- No pharmacological treatment
- Wild-type genotype
- Recorded on May 23, 2016 at 15:43:18

To fetch data only for a certain figure or experimental condition, we will define the following set of utilities that will allow us to filter only the NWB files we are interested in.

In [ ]:
from typing import Literal

# Session ID Parsing Functions
# These functions decode the rich metadata encoded in DANDI file paths

def get_session_id(asset_path: str) -> str:
    """
    Extract session ID from DANDI asset path.
    
    DANDI encodes paths as:
    sub-<subject_id>/sub-<subject_id>_ses-<session_id>_[desc-<description>]_<modalities>.nwb
    
    Example path:
    'sub-SubjectRecordedAt20160523154318/sub-SubjectRecordedAt20160523154318_ses-F3++SomExc++iSPN++OffState++none++WT++20160523154318_icephys.nwb'
    
    The session_id contains experimental metadata separated by '++':
    F3++SomExc++iSPN++OffState++none++WT++20160523154318
    │   │       │     │        │     │   │
    │   │       │     │        │     │   └─ Timestamp
    │   │       │     │        │     └─ Genotype (WT=Wild Type)  
    │   │       │     │        └─ Pharmacology (none=no drugs)
    │   │       │     └─ State (OffState=no stimulation)
    │   │       └─ Cell Type (iSPN=indirect pathway spiny projection neuron)
    │   └─ Measurement (SomExc=somatic excitability)
    └─ Figure (F3=Figure 3)
    """
    if not asset_path:
        return ""
    bottom_level_path = asset_path.split("/")[1]  # Remove top level subject
    session_id_with_ses_prefix = bottom_level_path.split("_")[1]
    session_id = session_id_with_ses_prefix.split("-")[1]

    return session_id

# Metadata extraction functions - each extracts a specific experimental parameter

def get_figure_number(session_id: str):
    """Extract which figure this data corresponds to (F1, F2, F3, etc.)"""
    return session_id.split("++")[0]

def is_figure_number(session_id: str, figure_number: Literal["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "SF3"]) -> bool:
    """Check if data belongs to a specific figure"""
    return get_figure_number(session_id) == figure_number


def get_measurement(session_id: str) -> str:
    """
    Extract measurement type:
    - SomExc: Somatic excitability (patch clamp at cell body)
    - DendExc: Dendritic excitability (patch clamp + 2-photon imaging)
    - DendSpine: Dendritic spine density measurements
    - BehavAIMs: Abnormal involuntary movement behavioral scoring
    - StriAChFP: Striatal acetylcholine fluorescent protein imaging
    """
    if not session_id:
        return ""
    return session_id.split("++")[1]

def is_measurement(session_id: str, measurement: Literal["SomExc", "DendExc", "DendSpine", "DendConfSpine", "DendOEPSC", "StriAChFP", "BehavAIMs", "BehavRot", "BehavVideo", "AChFP", "AIMs", "ConfSpine", "SpineDens", "oEPSC", "video"]) -> bool:
    """Filter data by measurement/experiment type"""
    return get_measurement(session_id) == measurement

def get_cell_type(session_id: str) -> str:
    """
    Extract cell type:
    - dSPN: Direct pathway spiny projection neurons
    - iSPN: Indirect pathway spiny projection neurons  
    - pan: Pan-neuronal (both types)
    """
    if not session_id:
        return ""
    return session_id.split("++")[2]

def is_cell_type(session_id: str, cell_type: Literal["dSPN", "iSPN", "pan"]) -> bool:
    """Filter data by cell type"""
    return get_cell_type(session_id) == cell_type

def get_state(session_id: str) -> str:
    """
    Extract experimental state:
    - CTRL: Control condition
    - PD: Parkinson's disease model
    - OffState: No levodopa treatment
    - OnState: With levodopa treatment
    """
    if not session_id:
        return ""
    return session_id.split("++")[3]

def is_state(session_id: str, state: Literal["CTRL", "LesionedControl", "OFF", "ON", "OffState", "OnState", "PD"]) -> bool:
    """Filter data by disease/treatment state"""
    return get_state(session_id) == state

def get_pharmacology(session_id: str) -> str:
    """
    Extract pharmacological treatment:
    - none: No drugs applied
    - CNO: Clozapine N-oxide (DREADD activator)
    - D1RaSch: D1 receptor agonist SCH23390
    - M1RaOxoM: M1 receptor agonist Oxotremorine-M
    """
    if not session_id:
        return ""
    return session_id.split("++")[4]

def is_pharmacology(session_id: str, pharmacology: Literal["none", "WT", "DMSO", "CNO", "D1RaSch", "D2RaSul", "M1RaOxoM", "M1RaThp", "M1RaTri"]) -> bool:
    """Filter data by pharmacological condition"""
    return get_pharmacology(session_id) == pharmacology

def get_genotype(session_id: str) -> str:
    """
    Extract animal genotype:
    - WT: Wild type
    - M1RCRISPR: M1 receptor knockout
    - iSPNM1RKO: iSPN-specific M1 receptor knockout
    """
    if not session_id:
        return ""
    return session_id.split("++")[5]

def is_genotype(session_id: str, genotype: Literal["CDGIKO", "M1RCRISPR", "WT", "iSPN", "iSPNM1RKO"]) -> bool:
    """Filter data by genotype"""
    return get_genotype(session_id) == genotype

def get_timestamp(session_id: str) -> str:
    """Extract recording timestamp (YYYYMMDDHHMMSS format)"""
    if not session_id:
        return ""
    return session_id.split("++")[6]

# Example: Parse metadata from a sample file
sample_asset = assets_list[2]
session_id = get_session_id(sample_asset.path)
figure_number = get_figure_number(session_id)
measurement = get_measurement(session_id)
cell_type = get_cell_type(session_id)
state = get_state(session_id)
pharmacology = get_pharmacology(session_id)
genotype = get_genotype(session_id)
timestamp = get_timestamp(session_id)

print("Parsed metadata from session ID:")
print(f"Session ID: {session_id}")
print(f"Figure: {figure_number}")
print(f"Measurement: {measurement}")
print(f"Cell Type: {cell_type}")
print(f"State: {state}")
print(f"Pharmacology: {pharmacology}")
print(f"Genotype: {genotype}")
print(f"Timestamp: {timestamp}")



## Current Clamp Data Access and Organization

### NWB Data Structure for Current Clamp Experiments

Current clamp data in NWB files is organized across several modules:
- **`acquisition`**: Raw voltage and current time series from individual recording sweeps
- **`intracellular_recordings`**: High-level table linking stimuli to responses  
- **`stimuli`**: Current injection protocols
- **`responses`**: Voltage recordings organized by experimental condition

Let's explore how to access and work with this hierarchical data organization using the filters we defined above to extract data from the somatic excitability experiments of figure 1.

In [ ]:
# Using the parsing functions to filter data
# Here we select somatic excitability experiments from Figure 1

# Define selection criteria using lambda function
def is_figure_number(session_id: str, figure_number: Literal["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "SF3"]) -> bool:
    """Check if data belongs to a specific figure"""
    return get_figure_number(session_id) == figure_number


def is_measurement(session_id: str, measurement: Literal["SomExc", "DendExc", "DendSpine", "DendConfSpine", "DendOEPSC", "StriAChFP", "BehavAIMs", "BehavRot", "BehavVideo", "AChFP", "AIMs", "ConfSpine", "SpineDens", "oEPSC", "video"]) -> bool:
    """Filter data by measurement/experiment type"""
    return get_measurement(session_id) == measurement

criteria = lambda asset: is_figure_number(get_session_id(asset.path), "F1") and is_measurement(get_session_id(asset.path), "SomExc")

# Filter the assets list to get only matching files
available_assets = [asset for asset in assets_list if criteria(asset)]

print(f"Found {len(available_assets)} somatic excitability assets from Figure 1:")
for i, asset in enumerate(available_assets[:3]):  # Show first 3 files
    session_id = get_session_id(asset.path)
    print(f"  {i+1}. {asset.path}")
    print(f"     Session: {session_id}")
    print(f"     Cell type: {get_cell_type(session_id)}")
    print(f"     State: {get_state(session_id)}")
    print(f"     Genotype: {get_genotype(session_id)}")
    print()

### Streaming NWB Files from DANDI

Here we demonstrate **streaming access** to NWB files directly from
DANDI's cloud storage. This approach enables:

- **No local downloads**: Access data directly over the internet without
storing files locally
- **Selective data access**: Read only the parts of the file you need
(lazy loading)
- **Memory efficiency**: Work with large datasets without loading
everything into memory
- **Global accessibility**: Access data from anywhere with an internet
connection

This is particularly valuable for large neurophysiology datasets that may
be too big to download entirely.

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO

asset = available_assets[0]
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()

nwbfile

### Exploring the Acquisition Module

The `acquisition` module contains the raw data streams recorded during the experiment. For current clamp experiments, this typically includes multiple `CurrentClampSeries` objects, each representing one recording sweep with a specific current injection.

In [ ]:
# The Current Clamp Responses are stored in acquisition. Let's check out the available keys
nwbfile.acquisition.keys()

### Accessing Individual Time Series

Each `CurrentClampSeries` in the acquisition module represents one recording sweep. Let's access and plot two different sweeps to see how individual time series are stored and retrieved. Notice how each series has its own timestamps and can be accessed independently.

In [ ]:
import matplotlib.pyplot as plt

# Create subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot CurrentClampSeries001 on the left
current_clamp_response_001 = nwbfile.acquisition["CurrentClampSeries001"]
timestamps_001 = current_clamp_response_001.get_timestamps()
unit_001 = current_clamp_response_001.unit

ax1.plot(timestamps_001, current_clamp_response_001.get_data_in_units())
ax1.set_xlabel("Time (s)")
ax1.set_ylabel(f"Voltage ({unit_001})")
ax1.set_title("CurrentClampSeries001")

# Plot CurrentClampSeries002 on the right
current_clamp_response_002 = nwbfile.acquisition["CurrentClampSeries002"]
timestamps_002 = current_clamp_response_002.get_timestamps()
unit_002 = current_clamp_response_002.unit

ax2.plot(timestamps_002, current_clamp_response_002.get_data_in_units())
ax2.set_xlabel("Time (s)")
ax2.set_ylabel(f"Voltage ({unit_002})")
ax2.set_title("CurrentClampSeries002")

plt.tight_layout()
plt.show()

Note that the timestamps of the units are different as they
are recorded at different times during the experiment. All the timestamps in NWB are aligned to the same time base (e.g., the start of the recording session). We will see how to align these timestamps in a same base for comparison further down

### Systematic Data Access via Intracellular Recordings Table

While individual sweeps can be accessed from the `acquisition` module, NWB provides a higher-level interface through the `intracellular_recordings` table. This table systematically organizes all stimulus-response pairs, making it easier to:

- Link current injection parameters to voltage responses
- Access metadata for each recording sweep
- Perform systematic analysis across all experimental conditions
- Build frequency-intensity (F-I) curves and other analyses

The table approach is particularly valuable when dealing with many recording sweeps with different experimental parameters.

In [ ]:
nwbfile.intracellular_recordings.to_dataframe()

We see in the table all the recordings, including their start and end times, as well as the associated metadata such as the electrode location and crucially the input current values.

In [ ]:
currents = nwbfile.intracellular_recordings["stimulus_current_pA"][:]

### Accessing Individual Sweep Data Through the Table

The `intracellular_recordings` table provides references to the actual time series data. Each row contains:
- **Stimulus parameters**: Current amplitude, timing, waveform
- **Response references**: Links to the voltage recordings in the acquisition module
- **Metadata**: Electrode properties, recording conditions

Let's extract one specific stimulus-response pair to see how this referencing system works.

In [ ]:
index = 20
current_in_pA = currents[index]
response_reference = nwbfile.intracellular_recordings["responses"]["response"][index]
response = response_reference.timeseries
response

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

number_of_samples = response.data.shape[0]
sampling_rate = response.rate
aligned_times = np.arange(number_of_samples) / sampling_rate
unit = response.unit
data_in_volts = response.get_data_in_units()
data_in_millivolts = data_in_volts * 1000  # Convert to millivolts

plt.plot(aligned_times, data_in_millivolts)
plt.xlabel("Time (s) aligned to response start")

plt.ylabel(f"Voltage (mV)")

### Systematic Access to All Sweeps

The real power of the `intracellular_recordings` table becomes apparent when accessing multiple sweeps systematically. Rather than manually tracking individual `CurrentClampSeries` names, we can iterate through the table to access all stimulus-response pairs.

Note the timestamp alignment: we create artificial timestamps using sampling rate and number of samples, effectively aligning all responses to stimulus onset (t=0). This is crucial for comparing responses across different current injection amplitudes.

In [ ]:
response_references = nwbfile.intracellular_recordings["responses"]["response"]

# Create a colormap for better visualization
colors = plt.cm.autumn_r(np.linspace(0, 1, len(currents)))

for index, (current_in_pA, response_reference) in enumerate(zip(currents, response_references)):
    # Skip every second plot for clarity
    if index % 2 == 0:
        response = response_reference.timeseries
        number_of_samples = response.data.shape[0]
        sampling_rate = response.rate
        data_in_volts = response.get_data_in_units()
        data_in_millivolts = data_in_volts * 1000  # Convert to millivolts  
        aligned_times = np.arange(number_of_samples) / sampling_rate
        plt.plot(aligned_times, data_in_millivolts, 
                label=f"{current_in_pA} pA", color=colors[index])

plt.xlabel("Time (s) aligned to response start")
plt.ylabel("Voltage (mV)")
plt.title("Voltage Responses to Stimulus Currents")
plt.legend()
plt.show()

We have skipped every second response to compress the map and use a sequential color map with yellow for the smallest current and red for the largest current to showcase the variation in voltage responses more clearly.

## Dendritic Excitability Experiments - Line Scans 

### NWB Organization for Two-Photon Imaging Data

Dendritic excitability experiments combine patch-clamp electrophysiology with two-photon microscopy. While the `acquisition` module also contains current clamp data (accessed the same way as above), we'll focus here on the optical physiology components:

**Data stored in `acquisition` module:**
- **Source Images**: Stored as `Images` container with individual `GrayscaleImage` objects
- **Line Scan Raw Data**: Stored as `TimeSeries` objects with time × pixel dimensions

**Data stored in `processing/ophys` module:**
- **ROI Responses**: Stored as `RoiResponseSeries` objects in a `Fluorescence` data interface
- **Plane Segmentation**: Stored as `PlaneSegmentation` tables containing pixel masks that define ROI locations

These experiments typically include multiple trials for both **distal** and **proximal** dendritic locations, allowing comparison of calcium responses at different distances from the soma.

In [ ]:
criteria = lambda asset: is_figure_number(get_session_id(asset.path), "F1") and is_measurement(get_session_id(asset.path), "DendExc")
available_assets = [asset for asset in assets_list if criteria(asset)]

### Filtering for Dendritic Excitability Data

First, we filter the assets to find dendritic excitability experiments from Figure 1. These files contain the two-photon imaging data combined with electrophysiology measurements.

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO

# Get one of the assets
asset = available_assets[0]
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")
io = NWBHDF5IO(file=file, load_namespaces=True)
nwbfile = io.read()
nwbfile

### Streaming a Dendritic Excitability NWB File

We'll stream one of the dendritic excitability files to explore how two-photon imaging data is organized in NWB. Notice how the same streaming approach works for both electrophysiology and imaging data.

### Accessing ROI Response Data from the Ophys Module

ROI responses are stored in the `processing/ophys/Fluorescence` data interface as `RoiResponseSeries` objects. Each series represents fluorescence changes over time for a specific region of interest (ROI).

Let's compare distal vs proximal dendritic responses by accessing multiple `RoiResponseSeries` objects. Notice the naming convention that allows us to programmatically identify different experimental conditions.

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt

roi_response_names = [
    "RoiResponseSeriesFluo4DistalDendrite1Trial001",
    "RoiResponseSeriesFluo4DistalDendrite1Trial002",
    "RoiResponseSeriesFluo4DistalDendrite1Trial003",
    "RoiResponseSeriesFluo4ProximalDendrite1Trial001",
    "RoiResponseSeriesFluo4ProximalDendrite1Trial002",
    "RoiResponseSeriesFluo4ProximalDendrite1Trial003",
]

ophys_module = nwbfile.processing['ophys']
fluoresence_module = ophys_module.data_interfaces["Fluorescence"]


roi_response_proximal = [fluoresence_module[name] for name in roi_response_names if "Proximal" in name]
roi_response_distal = [fluoresence_module[name] for name in roi_response_names if "Distal" in name]


color_distal, color_proximal = "tab:blue", "tab:orange"

plt.figure(figsize=(10, 6))

# Distal (filtered, one color; vary linestyle/alpha per trial)
for i, roi_response in enumerate(roi_response_distal, start=1):
    num_samples = roi_response.data.shape[0]
    sampling_rate = roi_response.rate
    aligned_timestamps = np.arange(num_samples) / sampling_rate
    b, a = butter(4, 10/(0.5*sampling_rate), btype="low")  # 10 Hz cutoff
    filtered_response = filtfilt(b, a, roi_response.data[:])
    plt.plot(
        aligned_timestamps, filtered_response,
        color=color_distal,
        label=f"Distal Trial{roi_response.name[-4:]}"
    )

# Proximal (filtered, one color; different style per trial)
for i, roi_response in enumerate(roi_response_proximal, start=1):
    num_samples = roi_response.data.shape[0]
    sampling_rate = roi_response.rate
    aligned_timestamps = np.arange(num_samples) / sampling_rate
    b, a = butter(4, 10/(0.5*sampling_rate), btype="low")  # 10 Hz cutoff
    filtered_response = filtfilt(b, a, roi_response.data[:])
    plt.plot(
        aligned_timestamps, filtered_response,
        color=color_proximal,
        label=f"Proximal Trial{roi_response.name[-4:]}"
    )

plt.xlabel("Time (s)")
plt.ylabel("Fluorescence (a.u.)")
plt.title("Distal vs Proximal ROI responses (low-pass 10 Hz)")
plt.legend()
plt.tight_layout()
plt.show()


### Line Scans and Fluoresence 

### Comprehensive Line Scan Visualization: Linking Images, Raw Data, and ROI Responses

This section demonstrates how to access and visualize the complete line scan data workflow in NWB:

1. **Source Images** (`GrayscaleImage` objects): Anatomical reference showing where line scans were performed
2. **Line Scan Raw Data** (`TimeSeries` objects): Raw fluorescence measurements over time and space  
3. **ROI Responses** (`RoiResponseSeries` objects): Processed fluorescence time series extracted from specific regions and are linked to them directly.

This demonstrates how NWB maintains the complete data provenance chain from raw measurements to processed analysis results.

In [ ]:
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.signal import butter, filtfilt

line_scan_raw_data_name = "TimeSeriesLineScanRawFluo4DistalDendrite1Trial001"
source_image_name = "ImageAlexa568DistalDendrite1Trial001"
roi_response_series_name = "RoiResponseSeriesFluo4DistalDendrite1Trial001"

line_scan_data = nwbfile.acquisition[line_scan_raw_data_name]
source_images_container = nwbfile.acquisition['ImageLineScanSource']
source_image = source_images_container[source_image_name]
fluoresence_module = ophys_module.data_interfaces["Fluorescence"]
roi_response = fluoresence_module[roi_response_series_name]

# Get line scan coordinates from the ROI response series
pixel_mask = roi_response.rois[0]["pixel_mask"].iloc[0]
               
print(f"✓ Extracted line scan coordinates:")
print(f"  X range: {pixel_mask['x'].min()} to {pixel_mask['x'].max()}")
print(f"  Y range: {pixel_mask['y'].min()} to {pixel_mask['y'].max()}")
print(f"  Number of pixels: {len(pixel_mask['x'])}")

# Define colors for different dyes
alexa568_color = '#FF4500'  # Red-orange color for Alexa Fluor 568 (structural dye)
fluo4_color = '#32CD32'     # Lime green color for Fluo-4 (looks good on white background)

# Create custom colormaps
alexa568_cmap = mcolors.LinearSegmentedColormap.from_list(
    "alexa568", ["black", alexa568_color], N=256
)
fluo4_cmap = mcolors.LinearSegmentedColormap.from_list(
    "fluo4", ["white", fluo4_color], N=256
)

# Get raw line scan data - no filtering
line_scan_raw_data = line_scan_data.data[:]
num_samples = line_scan_raw_data.shape[0]
num_pixels = line_scan_raw_data.shape[1]

print(f"Line scan data shape: {line_scan_raw_data.shape}")

# Get ROI response data and create filtered version
roi_response_data = roi_response.data[:]
roi_sampling_rate = roi_response.rate
print(f"ROI response shape: {roi_response_data.shape}")

# Simple first-order Butterworth filter for ROI response only
cutoff_freq = 10  # Hz - appropriate for calcium signals
b_roi, a_roi = butter(1, cutoff_freq/(0.5*roi_sampling_rate), btype="low")  # First order
roi_response_filtered = filtfilt(b_roi, a_roi, roi_response_data)

# Create figure with source image on left, raw data and fluorescence stacked on right
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(nrows=2, ncols=2, hspace=0.3, wspace=0.25)

# Left: Source image with line scan overlay (Alexa 568 - structural) - no inset
ax_src = fig.add_subplot(gs[:, 0])
ax_src.imshow(source_image, cmap=alexa568_cmap, aspect="equal", origin="upper")
ax_src.set_title("Source Image (Alexa 568) with Line Scan Overlay")
ax_src.set_xlabel("X position (pixels)")
ax_src.set_ylabel("Y position (pixels)")

ax_src.plot(pixel_mask["x"], pixel_mask["y"], "yellow", linewidth=3, alpha=0.9, label="Line scan")
ax_src.legend(loc="best")

# Top right: Raw line scan data with Fluo-4 colormap
ax_raw = fig.add_subplot(gs[0, 1])
im_raw = ax_raw.imshow(
    line_scan_raw_data.T,
    aspect="auto",
    cmap=fluo4_cmap,
    origin="lower",
    extent=[0, num_samples, 0, num_pixels],
)
ax_raw.set_title("Line Scan Raw Data (Fluo-4)")
ax_raw.set_xlabel("Line Scans (samples)")
ax_raw.set_ylabel("Position along line (pixels)")

# Ensure same x-limits for raw and ROI plots
ax_raw.set_xlim(0, num_samples)

# Colorbar for raw data - positioned well inside the plot
cax_raw = inset_axes(
    ax_raw,
    width="3%",
    height="50%",
    loc="upper right",
    bbox_to_anchor=(-0.15, -0.15, 1, 1),
    bbox_transform=ax_raw.transAxes,
    borderpad=0,
)
fig.colorbar(im_raw, cax=cax_raw, label="Ca²⁺ fluorescence")

# Bottom right: Raw and filtered ROI response time series
ax_roi = fig.add_subplot(gs[1, 1])
ax_roi.plot(roi_response_data, color=fluo4_color, linewidth=1, alpha=0.6, label="Raw")
ax_roi.plot(roi_response_filtered, color='darkgreen', linewidth=2, label=f"Filtered ({cutoff_freq} Hz, 1st order)")
ax_roi.set_title("ROI Response Time Series (Fluo-4)")
ax_roi.set_xlabel("Line Scans (samples)")
ax_roi.set_ylabel("Ca²⁺ fluorescence (a.u.)")
ax_roi.legend()

# Match x-limits to the raw line-scan plot
ax_roi.set_xlim(0, num_samples)

plt.tight_layout()
plt.show()

## Optogenetic Experiments - oEPSC Analysis

### NWB Organization for Optogenetic Data

Optogenetic experiments combine voltage clamp electrophysiology with optical stimulation to study synaptic responses. These experiments measure optogenetically-evoked postsynaptic currents (oEPSCs) in striatal neurons following ChR2-mediated activation of cortical terminals.

**Data stored in `acquisition` module:**
- **VoltageClampSeries**: Raw current recordings from voltage clamp experiments
- **OptogeneticSeries**: Optical stimulation parameters and timing (stored in `stimulus` module)

**Data stored in `intervals` module:**
- **optogenetic_epochs_table**: Detailed timing information for stimulation and detection windows
- **Stage definitions**: pre_stimulation, stimulation, detection, post_detection, inter_sweep_interval

### Filtering for Optogenetic Data from Figure 2

We use the same filtering approach to identify optogenetic experiments (oEPSC measurements) from Figure 2. These files contain voltage clamp recordings with optical stimulation protocols.

In [ ]:
# Filter for optogenetic experiments from Figure 2
criteria_oepsc = lambda asset: is_figure_number(get_session_id(asset.path), "F2") and is_measurement(get_session_id(asset.path), "oEPSC")

available_oepsc_assets = [asset for asset in assets_list if criteria_oepsc(asset)]

print(f"Found {len(available_oepsc_assets)} oEPSC assets from Figure 2:")
for i, asset in enumerate(available_oepsc_assets[:3]):  # Show first 3 files
    session_id = get_session_id(asset.path)
    print(f"  {i+1}. {asset.path}")
    print(f"     Session: {session_id}")
    print(f"     Cell type: {get_cell_type(session_id)}")
    print(f"     State: {get_state(session_id)}")
    print(f"     Genotype: {get_genotype(session_id)}")
    print()

### Streaming an  NWB file with Optogenetic stimulation

Let's stream one optogenetic file to explore the voltage clamp data organization and optogenetic stimulation protocols.

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO

asset = available_oepsc_assets[0]
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()
nwbfile

In [ ]:
print("Available acquisition keys:")
acquisition_keys = list(nwbfile.acquisition.keys())
print(acquisition_keys)

print(f"\nFound {len([k for k in acquisition_keys if 'VoltageClamp' in k])} VoltageClampSeries in acquisition module")
print("These contain the raw current recordings from voltage clamp experiments")

### Optogenetic Epochs Table

The key to understanding optogenetic experiments is the `optogenetic_epochs_table` stored in the `intervals` module. This table provides precise timing information for each experimental phase within each trial, including stimulation and detection windows.

**Column explanations:**
- **start_time/stop_time**: Precise timing boundaries for each epoch (seconds)
- **stimulation_on**: Boolean indicating if optical stimulation is active
- **pulse_length_in_ms**: Duration of each optical pulse (when stimulation_on=True)
- **period_in_ms**: Inter-pulse interval for pulse trains
- **number_pulses_per_pulse_train**: Number of pulses in train (1 for single pulses)
- **number_trains**: Number of pulse trains
- **intertrain_interval_in_ms**: Time between trains
- **power_in_mW**: Optical power delivered
- **optical_fiber_locations**: Description of fiber placement
- **stage_name**: Experimental phase (pre_stimulation, stimulation, detection, post_detection, inter_sweep_interval)

In [ ]:
# Access the optogenetic epochs table
optogenetics_table_df = nwbfile.intervals["optogenetic_epochs_table"].to_dataframe()
optogenetics_table_df.head()

### Visualizing Voltage Clamp Responses with Stimulation Windows

Let's plot two voltage clamp sweeps side-by-side, showing both the stimulation and detection windows that are crucial for oEPSC analysis. The optogenetic epochs table provides precise timing information for each experimental phase.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get optogenetic timing information
optogenetics_table_df = nwbfile.intervals["optogenetic_epochs_table"].to_dataframe()
stimulation_entries_df = optogenetics_table_df[optogenetics_table_df["stimulation_on"] == True]
detection_entries_df = optogenetics_table_df[optogenetics_table_df["stage_name"] == "detection"]

# Create subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

for trial_index, (trial_name, ax, title) in enumerate([(1, ax1, "VoltageClampSeriesSweep001"), (7, ax2, "VoltageClampSeriesSweep007")]):
    # Get voltage clamp response
    series_name = f"VoltageClampSeriesSweep{trial_index + 1:03d}"
    voltage_clamp_response = nwbfile.acquisition[series_name]
    timestamps_in_seconds = voltage_clamp_response.get_timestamps()
    data_in_amperes = voltage_clamp_response.get_data_in_units()
    data_in_pico_amperes = data_in_amperes * 1e12
    
    # Get timing info for this trial
    stimulation_info = stimulation_entries_df.iloc[trial_index]
    detection_info = detection_entries_df.iloc[trial_index]
    
    stimulation_start_ms = stimulation_info["start_time"] * 1000
    stimulation_stop_ms = stimulation_info["stop_time"] * 1000
    detection_start_ms = detection_info["start_time"] * 1000
    detection_stop_ms = detection_info["stop_time"] * 1000
    
    # Show data up to 50ms after detection window ends
    timestamps_in_milliseconds = timestamps_in_seconds * 1000
    starting_time = timestamps_in_milliseconds[0]
    end_time = detection_stop_ms + 50
    mask = (timestamps_in_milliseconds >= starting_time) & (timestamps_in_milliseconds <= end_time)
    timestamps_filtered = timestamps_in_milliseconds[mask]
    data_filtered = data_in_pico_amperes[mask]
    
    # Plot the data
    ax.plot(timestamps_filtered, data_filtered, 'b-', linewidth=1)
    ax.axvspan(stimulation_start_ms, stimulation_stop_ms, color="cyan", alpha=0.8, label="Stimulation")
    ax.axvspan(detection_start_ms, detection_stop_ms, color="gray", alpha=0.15, label="Detection")
    
    ax.set_xlabel("Time (ms)")
    ax.set_ylabel("Current (pA)")
    ax.set_title(title)
    
    if trial_index == 0:  # Add legend only to first subplot
        ax.legend()

plt.tight_layout()
plt.show()

print("Stimulation and detection windows:")
print(f"Trial 1 - Stimulation: {stimulation_entries_df.iloc[0]['start_time']*1000:.1f}-{stimulation_entries_df.iloc[0]['stop_time']*1000:.1f} ms")
print(f"Trial 1 - Detection: {detection_entries_df.iloc[0]['start_time']*1000:.1f}-{detection_entries_df.iloc[0]['stop_time']*1000:.1f} ms")

### Event Detection and Analysis

#### MAD-Based Noise Estimation for Robust Event Detection

For accurate oEPSC event detection, we use **Median Absolute Deviation (MAD)** instead of standard deviation to estimate baseline noise. This approach is more robust to outliers (including the events themselves) and provides better threshold estimates.

**Key parameters:**
- **Detection window shift**: 100ms after original detection start (to avoid stimulation artifacts)
- **Event threshold**: ±5 × MAD-based standard deviation from median
- **Event merging**: Combine events within 1ms (handles multi-threshold crossings from single events)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Control parameters for event detection
detection_window_shift_ms = 100  # Shift detection window start by this many ms
event_merge_distance_ms = 1.0    # Merge events within this distance

def merge_nearby_events(event_times, event_amplitudes, merge_distance_ms):
    """Merge events within merge_distance_ms, keeping maximum amplitude"""
    if len(event_times) == 0:
        return event_times, event_amplitudes
    
    times = np.array(event_times)
    amplitudes = np.array(event_amplitudes)
    sorted_indices = np.argsort(times)
    times = times[sorted_indices]
    amplitudes = amplitudes[sorted_indices]
    
    merged_times = []
    merged_amplitudes = []
    
    i = 0
    while i < len(times):
        current_time = times[i]
        current_amp = amplitudes[i]
        
        # Find all events within merge_distance_ms
        j = i + 1
        max_amp = current_amp
        max_amp_time = current_time
        
        while j < len(times) and (times[j] - current_time) <= merge_distance_ms:
            if amplitudes[j] > max_amp:
                max_amp = amplitudes[j]
                max_amp_time = times[j]
            j += 1
        
        merged_times.append(max_amp_time)
        merged_amplitudes.append(max_amp)
        i = j
    
    return merged_times, merged_amplitudes

# Demonstrate event detection on one sweep
trial_index = 0
series_name = f"VoltageClampSeriesSweep{trial_index + 1:03d}"
voltage_clamp_response = nwbfile.acquisition[series_name]
timestamps_in_seconds = voltage_clamp_response.get_timestamps()
data_in_amperes = voltage_clamp_response.get_data_in_units()
data_in_pico_amperes = data_in_amperes * 1e12

# Get timing information
stimulation_info = stimulation_entries_df.iloc[trial_index]
detection_info = detection_entries_df.iloc[trial_index]

detection_start_ms_original = detection_info["start_time"] * 1000
detection_stop_ms = detection_info["stop_time"] * 1000
detection_start_ms_shifted = detection_start_ms_original + detection_window_shift_ms

# Filter data to shifted detection window
timestamps_in_milliseconds = timestamps_in_seconds * 1000
detection_mask = (timestamps_in_milliseconds >= detection_start_ms_shifted) & (timestamps_in_milliseconds <= detection_stop_ms)
detection_data = data_in_pico_amperes[detection_mask]
detection_timestamps = timestamps_in_milliseconds[detection_mask]

if len(detection_data) > 0:
    # Calculate MAD-based standard deviation
    noise_median = np.median(detection_data)
    mad = np.median(np.abs(detection_data - noise_median))
    mad_std = mad * 1.4826  # MAD to standard deviation conversion
    
    event_threshold_positive = noise_median + 5 * mad_std
    event_threshold_negative = noise_median - 5 * mad_std
    
    # Find events
    positive_event_indices = np.where(detection_data > event_threshold_positive)[0]
    negative_event_indices = np.where(detection_data < event_threshold_negative)[0]
    
    # Calculate amplitudes and merge nearby events
    positive_event_times_raw = detection_timestamps[positive_event_indices] if len(positive_event_indices) > 0 else []
    negative_event_times_raw = detection_timestamps[negative_event_indices] if len(negative_event_indices) > 0 else []
    
    positive_event_amplitudes_raw = detection_data[positive_event_indices] - noise_median if len(positive_event_indices) > 0 else []
    negative_event_amplitudes_raw = noise_median - detection_data[negative_event_indices] if len(negative_event_indices) > 0 else []
    
    positive_event_times_merged, positive_event_amplitudes_merged = merge_nearby_events(
        positive_event_times_raw, positive_event_amplitudes_raw, event_merge_distance_ms)
    negative_event_times_merged, negative_event_amplitudes_merged = merge_nearby_events(
        negative_event_times_raw, negative_event_amplitudes_raw, event_merge_distance_ms)
    
    print(f"Event Detection Results for {series_name}:")
    print(f"  Noise median: {noise_median:.2f} pA")
    print(f"  MAD-based std: {mad_std:.2f} pA")
    print(f"  Positive threshold: {event_threshold_positive:.2f} pA")
    print(f"  Negative threshold: {event_threshold_negative:.2f} pA")
    print(f"  Raw events: +{len(positive_event_amplitudes_raw)}, -{len(negative_event_amplitudes_raw)}")
    print(f"  Merged events: +{len(positive_event_amplitudes_merged)}, -{len(negative_event_amplitudes_merged)}")
    
    if len(positive_event_amplitudes_merged) > 0:
        print(f"  Positive event amplitudes: {[f'{amp:.1f}' for amp in positive_event_amplitudes_merged]} pA")
    if len(negative_event_amplitudes_merged) > 0:
        print(f"  Negative event amplitudes: {[f'{amp:.1f}' for amp in negative_event_amplitudes_merged]} pA")
        
else:
    print(f"No data in detection window for {series_name}")

### Comprehensive Event Analysis Grid

Finally, let's analyze all voltage clamp sweeps in this session systematically, showing event detection across all trials in a grid format. This provides a complete overview of optogenetic responses throughout the experimental session.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get all acquisition keys for voltage clamp series
acquisition_keys = [key for key in nwbfile.acquisition.keys() if key.startswith("VoltageClampSeries")]
num_sweeps = len(acquisition_keys)

# Calculate grid dimensions
cols = 3
rows = (num_sweeps + cols - 1) // cols

# Create subplots grid
fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
axes = axes.flatten() if num_sweeps > 1 else [axes]

# Initialize summary statistics
all_positive_amplitudes = []
all_negative_amplitudes = []
total_events = 0

# Process each sweep
for i, sweep_key in enumerate(acquisition_keys):
    trial_number = int(sweep_key.split('Sweep')[-1])
    
    # Get voltage clamp response
    voltage_clamp_response = nwbfile.acquisition[sweep_key]
    timestamps_in_seconds = voltage_clamp_response.get_timestamps()
    data_in_amperes = voltage_clamp_response.get_data_in_units()
    data_in_pico_amperes = data_in_amperes * 1e12
    
    # Get timing information for this trial
    if trial_number <= len(stimulation_entries_df) and trial_number <= len(detection_entries_df):
        stimulation_info = stimulation_entries_df.iloc[trial_number - 1]
        detection_info = detection_entries_df.iloc[trial_number - 1]
        
        stimulation_start_ms = stimulation_info["start_time"] * 1000
        stimulation_stop_ms = stimulation_info["stop_time"] * 1000
        detection_start_ms_original = detection_info["start_time"] * 1000
        detection_stop_ms = detection_info["stop_time"] * 1000
        detection_start_ms_shifted = detection_start_ms_original + detection_window_shift_ms
        
        # Show data up to 50ms after detection window ends
        timestamps_in_milliseconds = timestamps_in_seconds * 1000
        starting_time = timestamps_in_milliseconds[0]
        end_time = detection_stop_ms + 50
        mask = (timestamps_in_milliseconds >= starting_time) & (timestamps_in_milliseconds <= end_time)
        timestamps_filtered = timestamps_in_milliseconds[mask]
        data_filtered = data_in_pico_amperes[mask]
        
        # Calculate noise statistics and detect events
        detection_mask = (timestamps_filtered >= detection_start_ms_shifted) & (timestamps_filtered <= detection_stop_ms)
        detection_data = data_filtered[detection_mask]
        detection_timestamps = timestamps_filtered[detection_mask]
        
        if len(detection_data) > 0:
            # MAD-based noise estimation
            noise_median = np.median(detection_data)
            mad = np.median(np.abs(detection_data - noise_median))
            mad_std = mad * 1.4826
            
            event_threshold_positive = noise_median + 5 * mad_std
            event_threshold_negative = noise_median - 5 * mad_std
            
            # Find and merge events
            positive_event_indices = np.where(detection_data > event_threshold_positive)[0]
            negative_event_indices = np.where(detection_data < event_threshold_negative)[0]
            
            positive_event_times_raw = detection_timestamps[positive_event_indices] if len(positive_event_indices) > 0 else []
            negative_event_times_raw = detection_timestamps[negative_event_indices] if len(negative_event_indices) > 0 else []
            
            positive_event_amplitudes_raw = detection_data[positive_event_indices] - noise_median if len(positive_event_indices) > 0 else []
            negative_event_amplitudes_raw = noise_median - detection_data[negative_event_indices] if len(negative_event_indices) > 0 else []
            
            # Merge nearby events
            positive_event_times_merged, positive_event_amplitudes_merged = merge_nearby_events(
                positive_event_times_raw, positive_event_amplitudes_raw, event_merge_distance_ms)
            negative_event_times_merged, negative_event_amplitudes_merged = merge_nearby_events(
                negative_event_times_raw, negative_event_amplitudes_raw, event_merge_distance_ms)
            
            # Collect amplitudes for summary
            all_positive_amplitudes.extend(positive_event_amplitudes_merged)
            all_negative_amplitudes.extend(negative_event_amplitudes_merged)
            
            sweep_events = len(positive_event_amplitudes_merged) + len(negative_event_amplitudes_merged)
            total_events += sweep_events
            
            # Plot
            ax = axes[i]
            ax.plot(timestamps_filtered, data_filtered, 'b-', linewidth=1)
            ax.axvspan(stimulation_start_ms, stimulation_stop_ms, color="cyan", alpha=0.8, label="Stimulation")
            ax.axvspan(detection_start_ms_original, detection_stop_ms, color="gray", alpha=0.05, label="Original Detection")
            ax.axvspan(detection_start_ms_shifted, detection_stop_ms, color="lightblue", alpha=0.15, label="Shifted Detection")
            ax.axhline(y=noise_median, color='black', linestyle='-', alpha=0.5, linewidth=1)
            ax.axhline(y=event_threshold_positive, color='red', linestyle='--', alpha=0.7, linewidth=1)
            ax.axhline(y=event_threshold_negative, color='red', linestyle='--', alpha=0.7, linewidth=1)
            
            # Get plot limits for event markers
            y_min, y_max = ax.get_ylim()
            
            # Add event markers
            for event_time in positive_event_times_merged:
                ax.plot([event_time, event_time], [event_threshold_positive, y_max], 
                        color='red', alpha=0.8, linewidth=2)
            
            for event_time in negative_event_times_merged:
                ax.plot([event_time, event_time], [y_min, event_threshold_negative], 
                        color='red', alpha=0.8, linewidth=2)
            
            ax.set_xlabel("Time (ms)")
            ax.set_ylabel("Current (pA)")
            ax.set_title(f"{sweep_key} (Events: {sweep_events})")
            
            # Add legend only to first subplot
            if i == 0:
                ax.legend(loc='upper right', fontsize=8)

# Hide empty subplots
for i in range(num_sweeps, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

# Print comprehensive summary
print(f"\n=== COMPREHENSIVE oEPSC ANALYSIS SUMMARY ===")
print(f"Session: {get_session_id(asset.path)}")
print(f"Total sweeps analyzed: {num_sweeps}")
print(f"Total events detected: {total_events}")
print(f"Positive events: {len(all_positive_amplitudes)}")
print(f"Negative events: {len(all_negative_amplitudes)}")

if len(all_positive_amplitudes) > 0:
    print(f"\nPositive event statistics:")
    print(f"  Mean amplitude: {np.mean(all_positive_amplitudes):.2f} ± {np.std(all_positive_amplitudes):.2f} pA")
    print(f"  Median amplitude: {np.median(all_positive_amplitudes):.2f} pA")
    print(f"  Range: {np.min(all_positive_amplitudes):.1f} to {np.max(all_positive_amplitudes):.1f} pA")

if len(all_negative_amplitudes) > 0:
    print(f"\nNegative event statistics:")
    print(f"  Mean amplitude: {np.mean(all_negative_amplitudes):.2f} ± {np.std(all_negative_amplitudes):.2f} pA")
    print(f"  Median amplitude: {np.median(all_negative_amplitudes):.2f} pA")
    print(f"  Range: {np.min(all_negative_amplitudes):.1f} to {np.max(all_negative_amplitudes):.1f} pA")

print(f"\nAnalysis parameters:")
print(f"  Detection window shift: {detection_window_shift_ms} ms")
print(f"  Event merge distance: {event_merge_distance_ms} ms")
print(f"  Event threshold: ±5 x MAD-based standard deviation")

# Acetylcholine Biosensor Experiments - GRABACh3.0 Imaging

## NWB Organization for Biosensor Data

GRABACh3.0 acetylcholine biosensor experiments measure cholinergic signaling dynamics using genetically encoded fluorescent sensors. These experiments reveal state-dependent modulation of acetylcholine release in Parkinson's disease and levodopa-induced dyskinesia.

**Key experimental parameters:**
- **Biosensor**: GRABACh3.0 genetically encoded acetylcholine indicator
- **Imaging**: Two-photon microscopy at 920nm excitation, 520nm emission  
- **Target region**: Dorsal striatum
- **Stimulation**: Electrical stimulation to evoke acetylcholine release
- **Experimental conditions**: Control (CTRL), Parkinsonian (PD/6-OHDA), Dyskinetic off-state (OFF)

**Data stored in `processing/ophys` module:**
- **ROI Response Series**: Processed fluorescence traces from regions of interest containing biosensor signal
- **Plane Segmentation**: ROI definitions showing areas of biosensor expression
- **Trials Table**: Experimental metadata including stimulation parameters, treatments, and timing

**Analysis workflow:**
1. Extract fluorescence time series from ROI response series
2. Apply background subtraction and baseline normalization
3. Calculate area under curve (AUC) for stimulus-evoked responses
4. Normalize using calibration values (Fmax from acetylcholine, Fmin from TTX)
5. Compare responses across experimental conditions

### Filtering for Acetylcholine Biosensor Data from Figure 5

These experiments examine acetylcholine release dynamics across three experimental conditions that model different stages of Parkinson's disease pathology:
- **CTRL**: Control unlesioned mice with normal cholinergic signaling
- **PD**: Parkinsonian mice (6-OHDA lesioned, no levodopa treatment) showing reduced cholinergic tone
- **OFF**: Dyskinetic mice in off-state (24-48h after chronic levodopa) with altered cholinergic dynamics

In [ ]:
# Filter for acetylcholine biosensor experiments from Figure 5
# Figure 5 examines acetylcholine release using GRABACh3.0 biosensor across disease states
criteria_oepsc = lambda asset: is_figure_number(get_session_id(asset.path), "F5")

available_oepsc_assets = [asset for asset in assets_list if criteria_oepsc(asset)]

print(f"Found {len(available_oepsc_assets)} acetylcholine biosensor assets from Figure 5:")

# Display metadata for each NWB file to understand experimental design
for i, asset in enumerate(available_oepsc_assets[:]):  
    session_id = get_session_id(asset.path)
    
    # Parse session ID components:
    # F5 = Figure 5, AChFP = Acetylcholine Fluorescent Protein (GRABACh3.0)
    # pan = pan-striatal expression, CTRL/PD/OFF = experimental condition
    print(f"  {i+1}. {asset.path}")
    print(f"     Session: {session_id}")
    print(f"     Cell type: {get_cell_type(session_id)}")  # Shows biosensor expression pattern
    print(f"     State: {get_state(session_id)}")          # CTRL, PD (6-OHDA), or OFF (dyskinetic)
    print(f"     Genotype: {get_genotype(session_id)}")    # WT mice with biosensor expression
    print()

### Streaming Acetylcholine Biosensor Data

The NWB files contain fluorescence time series from GRABACh3.0 biosensor experiments. Each file includes:
- **Trials table**: Metadata about stimulation timing, treatment conditions, and experimental parameters
- **ROI Response Series**: Processed fluorescence traces from regions of interest

In [ ]:
import h5py
import remfile
from pynwb import NWBHDF5IO

asset = available_oepsc_assets[0]
s3_url = asset.get_content_url(follow_redirects=1, strip_query=False)
file_system = remfile.File(s3_url)
file = h5py.File(file_system, mode="r")

io = NWBHDF5IO(file=file)
nwbfile = io.read()
nwbfile

We can extract the trials table as a data frame with the following method and inspect it.

In [ ]:
trials_df = nwbfile.trials.to_dataframe()
trials_df

And then we can see the structure of the session. Each trial is characterized by a treatment (control, quinpirole, sulpride, etc) a type of stimulation (single_pulse, burst or calibration) and also contains the name of the ROIResponseSeries that corresponds to this.

### Accessing Fluorescence Time Series Data


In [ ]:
fluorescence_module = nwbfile.processing["ophys"]["Fluorescence"]
fluorescence_module

In [ ]:
import matplotlib.pyplot as plt

trials_df = nwbfile.trials.to_dataframe()
trial_index = 10
trial_row = trials_df.iloc[trial_index]
stimulus_start_time = trial_row["stimulus_start_time"]
treatment = trial_row["treatment"]
roi_response_series_name = trial_row["roi_series_name"]
stimulation = trial_row["stimulation"]

roi_response_series = fluorescence_module[roi_response_series_name]


timestamps = roi_response_series.get_timestamps()
data = roi_response_series.data[:]

plt.plot(timestamps, data)
plt.axvline(stimulus_start_time, color='r', linestyle='--')
plt.title(f"{treatment} - {stimulation}")
plt.ylabel("Fluorescence (a.u.)")
plt.xlabel("Time (s)")

In [ ]:
import numpy as np

# pick 6 random trials that have a stimulus_start_time
valid_idx = trials_df[trials_df["stimulus_start_time"].notna()].index.values
rng = np.random.default_rng(42)
chosen = rng.choice(valid_idx, size=6, replace=False)

fig, axes = plt.subplots(2, 3, figsize=(15, 8), sharex=False, sharey=True)
axes = axes.ravel()

# compute global y-limits across the chosen series (skip missing)
y_mins, y_maxs = [], []
for tid in chosen:
    row = trials_df.loc[tid]
    roi_name = row["roi_series_name"]
    series = fluorescence_module[roi_name]

    y = series.data[:]
    y_mins.append(np.nanmin(y))
    y_maxs.append(np.nanmax(y))

ymin = float(np.min(y_mins))
ymax = float(np.max(y_maxs))
pad = 0.05 * (ymax - ymin) if ymax > ymin else 1.0
ymin -= pad
ymax += pad

for ax, tid in zip(axes, chosen):
    row = trials_df.loc[tid]
    roi_name = row["roi_series_name"]
    stim_t = float(row["stimulus_start_time"])
    series = fluorescence_module[roi_name]

    ts = series.get_timestamps()
    y = series.data[:]
    rel_ts = ts - stim_t

    ax.plot(rel_ts, y, color="C0", lw=1)
    ax.axvline(0.0, color="r", linestyle="--", lw=1)
    ax.set_title(f"Observation {tid}: {row['treatment']} — {row['stimulation']}", fontsize=10)
    ax.set_xlabel("Time from stimulus (s)")
    ax.set_ylabel("Fluorescence (a.u.)")
    ax.set_ylim(ymin, ymax)

# overall layout
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()